In [1]:
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.svm import NuSVR
from sklearn.base import clone
from sklearn.model_selection import KFold
from sklearn.preprocessing import MaxAbsScaler
from sklearn.metrics import max_error, mean_absolute_error, mean_squared_error, median_absolute_error, r2_score
from collections.abc import Iterable
from gtm_steps import MultiLabelGTM

In [2]:
class ModelStrategy(object):
    def __init__(self, regressor) -> None:
        self._regressors = regressor

    def get_regressors(self):
        return self._regressors

    def set_strategy(self, regressor) -> None:
        self._regressors = regressor

    def fit(self, X_train, Y_train) -> None:
        self._fitted_regressors = []
        if not isinstance(self._regressors, Iterable):
            # Single regressors for all
            for y_train in Y_train.T:
                regressor = clone(self._regressors, safe=True)
                # Fit and save all regressors
                regressor.fit(X_train, y_train)
                self._fitted_regressors.append(regressor)
        else:
            assert len(self._regressors) == len(Y_train.T), \
                "Length of regressor should be the same as outputs in data"
            # Multiple regressors
            for _regressor, y_train in zip(self._regressors, Y_train.T):
                regressor = clone(_regressor, safe=True)
                # Fit and save all regressors
                regressor.fit(X_train, y_train)
                self._fitted_regressors.append(regressor)

        self._regressors = self._fitted_regressors

    def fit_transform(self, X_train, Y_train):
        # Fit on data
        self.fit(X_train, Y_train)
        # Transfrom data to numpy array and return
        return self.transform(X_train)

    def transform(self, X):
        y_pred = []
        for regressor in self._fitted_regressors:
            # For each saved regressor predict an output value
            y_pred.append(regressor.predict(X))
        return np.stack(y_pred, axis=1)


class AdapterSGTM(object):
    def __init__(self, number_of_steps = 1000, verbose=0):
        self._gtm = MultiLabelGTM(number_of_steps = number_of_steps, center_of_mass = True, verbose=verbose)

    def fit(self, X_train) -> None:
        self._gtm.fit(X_train, X_train)

    def fit_transform(self, X_train):
        self._gtm.fit(X_train, X_train)
        return self._gtm.predict(X_train)

    def transform(self, X):
        return self._gtm.predict(X)


def relative_root_mean_squared_error(y, y_pred):
    n = len(y) # update
    num = np.sum(np.square(y - y_pred)) / n  # update
    den = np.sum(np.square(y_pred))
    squared_error = num / den
    rrmse_loss = np.sqrt(squared_error)
    return rrmse_loss

def calculate_errors(y, y_pred):
    MaxError = max_error                         (y, y_pred)
    MedError = median_absolute_error             (y, y_pred)
    MAE = mean_absolute_error                    (y, y_pred)
    MSE = mean_squared_error                     (y, y_pred)
    RMSE = mean_squared_error                    (y, y_pred, squared=False)
    RRMSE = relative_root_mean_squared_error     (y, y_pred)
    R2 = r2_score                                (y, y_pred)

    return {
            'MaxError' : MaxError,
            'MedError' : MedError,
            'MAE' :      MAE,
            'MSE' :      MSE,
            'RMSE' :     RMSE,
            'RRMSE' :    RRMSE,
            'R2' :       R2}

In [3]:
# Load all data with pandas
data = pd.read_csv('/content/Tunneling_Induced_building_damage_dataset_V2.txt', sep='\t')
data = data.drop(labels = 'Number', axis=1)

data = data.drop(labels = [
    'Identifier',
    'Unnamed: 17',
    'Unnamed: 19',
    'Unnamed: 21',
    'Unnamed: 22',
    'Unnamed: 27',
    'Unnamed: 27',
    'Unnamed: 28',
    'Unnamed: 29',
    'Unnamed: 30',
    'Unnamed: 31'
], axis=1)

# remove outliers
data = data[(np.abs(stats.zscore(data.select_dtypes(exclude='object'))) < 3).all(axis=1)]
data_columns = data.columns
data.head()
X = data.iloc[:,:15].to_numpy()
Y = data[['C. Width', 'Tot. Cracks ', 'L. Average', 'Slope', 'Tilt', 'Ang. Dist.', 'Max Strain', 'G. Average']]
target_columns = Y.columns
Y = Y.to_numpy()

In [4]:
import sklearn

best_params = {
 'C. Width':     sklearn.svm.NuSVR(nu=0.7381023173986698, gamma=0.45626350309874447),
 'Tot. Cracks ': sklearn.svm.NuSVR(nu=0.830442776799938, gamma=0.4700662946537708),
 'L. Average':   sklearn.svm.NuSVR(nu=0.6280090013938833, gamma=0.7299380533150713),
 'Slope':        sklearn.svm.NuSVR(nu=0.9845962774238791, gamma=0.6396624525695515),
 'Tilt':         sklearn.svm.NuSVR(nu=0.8866630207138445, gamma=0.6432339945867557),
 'Ang. Dist.':   sklearn.svm.NuSVR(nu=0.15206383918813196, gamma=0.20058517935067138),
 'Max Strain':   sklearn.svm.NuSVR(nu=0.4029207009764526, gamma=0.794107880754981),
 'G. Average':   sklearn.svm.NuSVR(nu=0.9531355996926897, gamma=0.6351556537913738),
}

In [8]:
model_1_errors_test = []
model_2_errors_test = []

kf = KFold(n_splits=5, shuffle=True, random_state=42)

for train_index, test_index in kf.split(X, Y):
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]

    scaler_x = MaxAbsScaler()
    scaler_y = MaxAbsScaler()

    model1 = ModelStrategy(NuSVR())
    model2 = AdapterSGTM(verbose=1)
    model3 = ModelStrategy(best_params.values())

    X_train = scaler_x.fit_transform(X_train)
    Y_train_scaled = scaler_y.fit_transform(Y_train)
    X_test = scaler_x.transform(X_test)

    # Model 1 fit on the model
    Y_train_pred = model1.fit_transform(X_train, Y_train_scaled)
    Y_test_pred = model1.transform(X_test)

    # Use decomposition to increase input features by merge X_train & Y_pred
    X_train_new = model2.fit_transform(np.concatenate((X_train, Y_train_pred), axis=1))
    X_test_new = model2.transform(np.concatenate((X_test, Y_test_pred), axis=1))

    # Model 2 fit on the model
    Y_train_pred_new = model3.fit_transform(X_train_new, Y_train_scaled)
    Y_test_pred_new = model3.transform(X_test_new)

    for i, target_name in enumerate(target_columns):
      # Model 1 (Save results for calculating errors)
      model_1_errors_test.append({f'{target_name}' : calculate_errors(Y_test.T[i], scaler_y.inverse_transform(Y_test_pred).T[i])})
      # Model 2 (Save results for calculating errors)
      model_2_errors_test.append({f'{target_name}' : calculate_errors(Y_test.T[i], scaler_y.inverse_transform(Y_test_pred_new).T[i])})

Total input features: 23
--- 0.09115815162658691 seconds ---
Total input features: 23
--- 0.024648666381835938 seconds ---
Total input features: 23
--- 0.0281369686126709 seconds ---
Total input features: 23
--- 0.024341106414794922 seconds ---
Total input features: 23
--- 0.025191307067871094 seconds ---


In [9]:
class AveragesCalculator:
    def __init__(self, data):
        self.data = data

    def _split_data(self):
        splitted_data = {}

        for dictionary in self.data:
            for key, inner_dict in dictionary.items():
                if key not in splitted_data:
                    splitted_data[key] = []
                splitted_data[key].append(inner_dict)

        return splitted_data

    def _sum_values(self, data):
        sums = {}

        for key, values in data.items():
            sums[key] = {}
            num_elements = len(values)
            for dictionary in values:
                for inner_key, value in dictionary.items():
                    if inner_key in sums[key]:
                        sums[key][inner_key] += value
                    else:
                        sums[key][inner_key] = value
            # Store the number of elements for each key
            sums[key]['num_elements'] = num_elements

        return sums

    def _compute_averages(self, sums):
        averages = {}

        for key, value in sums.items():
            averages[key] = {}
            num_elements = value.pop('num_elements')
            # Remove num_elements from the dictionary
            for inner_key, inner_value in value.items():
                averages[key][inner_key] = inner_value / num_elements

        return averages

    def calculate_averages(self):
        splitted_data = self._split_data()
        sums = self._sum_values(splitted_data)
        averages = self._compute_averages(sums)

        return averages


def add_sufix_to_keys(dictionary, sufix):
    return {key + sufix : value for key, value in dictionary.items()}

In [12]:
# Calculate the averages
_model_1_errors_test = AveragesCalculator(model_1_errors_test).calculate_averages()
results = pd.DataFrame(_model_1_errors_test)
results.loc['MaxError'] = results.loc['MaxError'].apply(round, ndigits=3)
results.loc['MedError'] = results.loc['MedError'].apply(round, ndigits=4)
results.loc['MAE']      = results.loc['MAE'].apply(round, ndigits=4)
results.loc['MSE']      = results.loc['MSE'].apply(round, ndigits=8)
results.loc['RMSE']     = results.loc['RMSE'].apply(round, ndigits=4)
results.loc['RRMSE']    = results.loc['RRMSE'].apply(round, ndigits=2)
results.loc['R2']       = results.loc['R2'].apply(round, ndigits=2)
results.to_excel(f'NuSVRBasicResults.xlsx')
results

C. Width  Tot. Cracks   L. Average         Slope          Tilt  \
MaxError  9.707000      8.347000    6.751000  3.000000e-03  3.000000e-03   
MedError  1.270400      0.883300    0.929500  2.000000e-04  2.000000e-04   
MAE       1.752400      1.268200    1.236700  3.000000e-04  3.000000e-04   
MSE       5.994345      3.260914    2.782104  2.200000e-07  2.100000e-07   
RMSE      2.441600      1.801400    1.665300  5.000000e-04  5.000000e-04   
RRMSE     0.040000      0.040000    0.030000  2.000000e-02  2.000000e-02   
R2        0.660000      0.730000    0.780000  8.900000e-01  8.800000e-01   

            Ang. Dist.    Max Strain    G. Average  
MaxError  1.000000e-03  3.000000e-03  2.000000e-03  
MedError  1.000000e-04  3.000000e-04  2.000000e-04  
MAE       1.000000e-04  4.000000e-04  2.000000e-04  
MSE       3.000000e-08  2.600000e-07  1.100000e-07  
RMSE      2.000000e-04  5.000000e-04  3.000000e-04  
RRMSE     3.000000e-02  4.000000e-02  2.000000e-02  
R2        7.600000e-01  6.700000e-01  8.800000e-01

In [11]:
# Calculate the averages
_model_2_errors_test = AveragesCalculator(model_2_errors_test).calculate_averages()
results = pd.DataFrame(_model_2_errors_test)
results.loc['MaxError'] = results.loc['MaxError'].apply(round, ndigits=3)
results.loc['MedError'] = results.loc['MedError'].apply(round, ndigits=4)
results.loc['MAE']      = results.loc['MAE'].apply(round, ndigits=4)
results.loc['MSE']      = results.loc['MSE'].apply(round, ndigits=8)
results.loc['RMSE']     = results.loc['RMSE'].apply(round, ndigits=4)
results.loc['RRMSE']    = results.loc['RRMSE'].apply(round, ndigits=2)
results.loc['R2']       = results.loc['R2'].apply(round, ndigits=2)
results.to_excel(f'NuSVRBestResults.xlsx')
results

C. Width  Tot. Cracks   L. Average         Slope          Tilt  \
MaxError  9.530000      8.164000    6.077000  3.000000e-03  3.000000e-03   
MedError  1.041200      0.756700    0.799300  2.000000e-04  2.000000e-04   
MAE       1.608700      1.158700    1.141600  3.000000e-04  3.000000e-04   
MSE       5.398013      2.971854    2.541116  2.000000e-07  2.000000e-07   
RMSE      2.318900      1.716100    1.590000  4.000000e-04  4.000000e-04   
RRMSE     0.040000      0.030000    0.030000  1.000000e-02  2.000000e-02   
R2        0.700000      0.760000    0.800000  9.000000e-01  8.900000e-01   

            Ang. Dist.    Max Strain    G. Average  
MaxError  1.000000e-03  3.000000e-03  2.000000e-03  
MedError  1.000000e-04  2.000000e-04  1.000000e-04  
MAE       1.000000e-04  3.000000e-04  2.000000e-04  
MSE       3.000000e-08  2.300000e-07  9.000000e-08  
RMSE      2.000000e-04  5.000000e-04  3.000000e-04  
RRMSE     3.000000e-02  4.000000e-02  2.000000e-02  
R2        8.000000e-01  7.000000e-01  8.900000e-01